# Import and Setup the model

In [ ]:
from agents import Agent, OpenAIChatCompletionsModel, AsyncOpenAI, Runner, trace, function_tool
from dotenv import load_dotenv
import requests
import os
import asyncio
import sendgrid
from sendgrid.helpers.mail import Email, To, Content, Mail
from typing import Dict
from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from agents.model_settings import ModelSettings

In [ ]:
load_dotenv(override=True)

In [ ]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_BASE_URL = os.getenv("GEMINI_BASE_URL")

In [ ]:
gemini_client = AsyncOpenAI(
    api_key = GEMINI_API_KEY,
    base_url = GEMINI_BASE_URL
)

model = OpenAIChatCompletionsModel(
    model = 'gemini-2.0-flash',
    openai_client = gemini_client
)

# Search Agent

In [ ]:

@function_tool
def google_search(query: str) -> str:
    """
    Perform a Google search using Custom Search API and return top results as text.
    """
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "q": query,
        "num": 5  # top 5 results
    }

    try:
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        data = resp.json()
        results = []
        for item in data.get("items", []):
            title = item.get("title", "")
            snippet = item.get("snippet", "")
            link = item.get("link", "")
            results.append(f"{title}: {snippet} ({link})")
        return "\n".join(results)
    except Exception as e:
        return f"Google search failed: {e}"


In [ ]:
INSTRUCTIONS = """
You are a research assistant AI. Given a topic, you search the web and produce a concise, structured summary
capturing the key points. Follow these rules:

1. Summarize in 2-3 paragraphs, under 300 words.
2. Focus on the most relevant and recent information.
3. Include key frameworks, trends, or technologies if applicable.
4. Present findings in short, clear sentences or bullet points if helpful.
5. Ignore any unrelated details or fluff.
6. Do not add personal opinions, commentary, or conclusions.

Output should be easy to read and suitable for use in research reports.
"""

In [ ]:
search_agent = Agent(
    name="Deep Research Agent",
    instructions=INSTRUCTIONS,
    tools=[google_search], 
    model=model,
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

# Planner Agent

In [ ]:
HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchItem(BaseModel):
    reason: str
    "You reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name = 'PlannerAgent',
    instructions = INSTRUCTIONS,
    model = model,
    output_type=WebSearchPlan,
)

In [ ]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

# Send Email Tool

In [ ]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("deepnagpal147514@gmail.com")  # Verified sender
    to_email = To("akagamishanks1554@gmail.com")      # Recipient

    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()

    response = sg.client.mail.send.post(request_body=mail)

    return {
        "status": "success" if response.status_code == 202 else "failed",
        "code": response.status_code,
        "body": response.body.decode() if response.body else "",
    }


In [ ]:
send_email

# Email Agent

In [ ]:
INSTRUCTIONS = """
    You are able to send a nicely formatted HTML email based on a detailed report.
    You will be provided with a detailed report. You should use your tool to send one email, providing the 
    report converted into clean, well presented HTML with an appropriate subject line.
"""

In [ ]:
email_agent = Agent(
    name = 'Email Agent',
    instructions = INSTRUCTIONS,
    tools = [send_email],
    model = model
)

# Writer Agent

In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 4-5 pages of content, at least 500 words."
)

In [ ]:
class ReportData(BaseModel):
    short_summary: str = Field(description = "A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description = "The Final Report")

    follow_up_questions: list[str] = Field(description = "Suggested topics to research further")

In [ ]:
writer_agent = Agent(
    name = 'Writer Agent',
    instructions = INSTRUCTIONS,
    model = model,
    output_type = ReportData
)

# 3 Function to plan and execute the search using planner_agent & search_agent

In [ ]:
async def plan_search(query: str):
    """ Use the `planner_agent` to plan which searches to run for query. """
    print("Planning Searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

In [ ]:
async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search paln"""
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [ ]:
async def perform_searches(search_plan:  WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finish Searching")
    return results

# 2 functions to write a report and email it

In [ ]:
async def write_report(query:  str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results. """
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

In [ ]:
async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report. """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

# Showtime!

In [ ]:
query = "Future of Artificial Intelligence"

In [ ]:
with trace("Research Tace"):
    print("Searching research...")
    search_plan = await plan_search(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)
    print(report)
    print("Hooray!")